<a href="https://colab.research.google.com/github/bsse1027/Stock_Indicators_Calculator/blob/main/Grab_Company.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import requests
import csv
import array as arr
from pprint import pprint


In [45]:
def getCompanyCodeList():
  response = requests.get("https://www.amarstock.com/LatestPrice/34267d8d73dd?fbclid=IwAR0UNljsm-ezbNkKryoHblOkrZNNzdjUGad6lcqQEydQbKuP7TRbZHYOFr4")
  response.raise_for_status()
  if response.status_code == 200:
    todos = json.loads(response.text)
    all_company_list = []
    for x in todos:
      companyId = x['Scrip']
      all_company_list.append(companyId)
  return all_company_list

In [48]:
len(getCompanyCodeList())

388

In [58]:
import pandas as pd
import json

#def getMovingAverage(companyId):
companyId = "ABBANK"
response2 = requests.get(f"https://www.amarstock.com/data/afe01cd8b512070a/?scrip={companyId}&cycle=Day1&dtFrom=2016-12-15T05%3A02%3A13.318Z&fbclid=IwAR0qZBhgiqSV6L6xTerlCEsXvVwtaLMaQvTqqMfUmjloMfBO2jocwV95DE8")
if (response2.status_code == 200):  
  todos2 = json.loads(response2.text)
json_string = json.dumps(todos2)
df = pd.read_json(json_string)
# for i in range(len(df)):
#   if df.loc[i, 'DateEpoch'] == 1483228800000:
#     print(f"{df.loc[i, 'DateString']}")
close_p = df[['Close', 'DateString']].copy()
close_p['SMA12'] = close_p['Close'].rolling(12).mean()
final_list = close_p[['DateString', 'SMA12']].copy()
final_list

,DateString,SMA12
0,15/12/2016 00:00:00,NaN
1,18/12/2016 00:00:00,NaN
2,19/12/2016 00:00:00,NaN
3,20/12/2016 00:00:00,NaN
4,21/12/2016 00:00:00,NaN
...,...,...
1355,25/09/2022 00:00:00,10.025000
1356,26/09/2022 00:00:00,10.000000
1357,27/09/2022 00:00:00,9.975000
1358,28/09/2022 00:00:00,9.941667


In [31]:
companyId = "BEXIMCO"
getMovingAverage(companyId)

(1360, 14)


In [78]:
import pandas as pd
import json


#def getMACD(companyId):
companyId = "DUTCHBANGL"
response2 = requests.get(f"https://www.amarstock.com/data/afe01cd8b512070a/?scrip={companyId}&cycle=Day1&dtFrom=2016-12-15T05%3A02%3A13.318Z&fbclid=IwAR0qZBhgiqSV6L6xTerlCEsXvVwtaLMaQvTqqMfUmjloMfBO2jocwV95DE8")
if (response2.status_code == 200):  
  todos2 = json.loads(response2.text)
json_string = json.dumps(todos2)
df = pd.read_json(json_string)
# for i in range(len(df)):
#   if df.loc[i, 'DateEpoch'] == 1483228800000:
#     print(f"{df.loc[i, 'DateString']}")
close_p = df[['Close', 'DateString']].copy()
# Get the 26-day EMA of the closing price
k = close_p['Close'].ewm(span=12, adjust=False).mean()
# Get the 12-day EMA of the closing price
d = close_p['Close'].ewm(span=26, adjust=False).mean()
# Subtract the 26-day EMA from the 12-Day EMA to get the MACD
macd = k - d
# Get the 9-Day EMA of the MACD for the Trigger line
macd_s = macd.ewm(span=9, adjust=False).mean()
# Calculate the difference between the MACD - Trigger for the Convergence/Divergence value
macd_h = macd - macd_s
# Add all of our new values for the MACD to the dataframe
close_p['macd'] = df.index.map(macd)
close_p['macd_h'] = df.index.map(macd_h)
close_p['macd_s'] = df.index.map(macd_s)
close_p


,Close,DateString,macd,macd_h,macd_s
0,34.687747,15/12/2016 00:00:00,0.000000,0.000000,0.000000
1,34.308300,18/12/2016 00:00:00,-0.030269,-0.024215,-0.006054
2,34.371542,19/12/2016 00:00:00,-0.048595,-0.034033,-0.014562
3,34.150198,20/12/2016 00:00:00,-0.080055,-0.052395,-0.027661
4,34.213439,21/12/2016 00:00:00,-0.098747,-0.056869,-0.041878
...,...,...,...,...,...
1354,63.800000,25/09/2022 00:00:00,-0.005660,-0.200838,0.195177
1355,63.800000,26/09/2022 00:00:00,-0.056377,-0.201244,0.144866
1356,63.300000,27/09/2022 00:00:00,-0.135356,-0.224178,0.088822
1357,63.000000,28/09/2022 00:00:00,-0.219623,-0.246756,0.027133


In [76]:
import pandas as pd
import json

#def getRSI(companyId):
companyId = "BANKASIA"
response2 = requests.get(f"https://www.amarstock.com/data/afe01cd8b512070a/?scrip={companyId}&cycle=Day1&dtFrom=2016-12-15T05%3A02%3A13.318Z&fbclid=IwAR0qZBhgiqSV6L6xTerlCEsXvVwtaLMaQvTqqMfUmjloMfBO2jocwV95DE8")
if (response2.status_code == 200):  
  todos2 = json.loads(response2.text)
json_string = json.dumps(todos2)
df = pd.read_json(json_string)
# for i in range(len(df)):
#   if df.loc[i, 'DateEpoch'] == 1483228800000:
#     print(f"{df.loc[i, 'DateString']}")
close_p = df[['Close', 'DateString']].copy()
# Calculate Price Differences
close_p['diff'] = close_p['Close'].diff(1)
# Calculate Avg. Gains/Losses
close_p['gain'] = close_p['diff'].clip(lower=0).round(2)
close_p['loss'] = close_p['diff'].clip(upper=0).abs().round(2)

window_length = 14
# Get initial Averages
close_p['avg_gain'] = close_p['gain'].rolling(window=window_length, min_periods=window_length).mean()[:window_length+1]
close_p['avg_loss'] = close_p['loss'].rolling(window=window_length, min_periods=window_length).mean()[:window_length+1]


# Get WMS averages
# Average Gains
for i, row in enumerate(close_p['avg_gain'].iloc[window_length+1:]):
    close_p['avg_gain'].iloc[i + window_length + 1] =\
        (close_p['avg_gain'].iloc[i + window_length] *
         (window_length - 1) +
         close_p['gain'].iloc[i + window_length + 1])\
        / window_length
# Average Losses
for i, row in enumerate(close_p['avg_loss'].iloc[window_length+1:]):
    close_p['avg_loss'].iloc[i + window_length + 1] =\
        (close_p['avg_loss'].iloc[i + window_length] *
         (window_length - 1) +
         close_p['loss'].iloc[i + window_length + 1])\
        / window_length
# Calculate RS
close_p['rs'] = close_p['avg_gain'] / close_p['avg_loss']
# Calculate RSI
close_p['rsi'] = 100 - (100 / (1.0 + close_p['rs']))
final_rsi = close_p[['DateString', 'rsi']].copy()
final_rsi

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,DateString,rsi
0,15/12/2016 00:00:00,NaN
1,18/12/2016 00:00:00,NaN
2,19/12/2016 00:00:00,NaN
3,20/12/2016 00:00:00,NaN
4,21/12/2016 00:00:00,NaN
...,...,...
1355,25/09/2022 00:00:00,48.638124
1356,26/09/2022 00:00:00,48.638124
1357,27/09/2022 00:00:00,48.638124
1358,28/09/2022 00:00:00,48.638124
